In [4]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_community.document_loaders import PyPDFLoader
loaders = [
    PyPDFLoader("./documents/pool_doc.pdf"),
    PyPDFLoader("./documents/pool_doc.pdf"),
    PyPDFLoader("./documents/pool_care_handbook.pdf"),
    PyPDFLoader("./documents/pool_operations_guide.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [7]:
splits = text_splitter.split_documents(docs)
len(splits)

506

In [8]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\mbari\AppData\Local\Temp\ipykernel_11656\2848396342.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings()


In [9]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [10]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [12]:
import numpy as np

In [13]:
np.dot(embedding1, embedding2)

0.9630350414845885

In [14]:
np.dot(embedding1, embedding3)

0.7701147991091332

In [15]:
np.dot(embedding2, embedding3)

0.759113000017713

In [16]:
np.dot(embedding3, embedding2)

0.759113000017713

In [17]:
from langchain.vectorstores import Chroma

In [18]:
persist_directory = "documents/chroma/"

In [21]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [23]:
print(vectordb._collection.count())

506


In [24]:
question = "what is CYA"

In [25]:
docs = vectordb.similarity_search(question, k=3)

In [26]:
len(docs)

3

In [30]:
print(docs[0].page_content)

CONDITIONER / CHLORINE STABILIZER :  Also called Cyanuric Acid 
(CYA) or stabilizer, this chemical provides a shield from the sun aro und 
the chlorine molecule, extending the efﬁcacy and saving you mone y.
CORROSION :  The effects of a acidic pool environment—one in which 
the pH and/or alkalinity are very low.   Corrosion results in the form of 
etching, pitting, or erosion of pool equipment and surfaces.  
COUPLING :  A plumbing ﬁtting designed to join two pieces of pipe.
COVER, AUTOMATIC :  Solid reinforced vinyl which rolls onto a reel on 
one end of the pool and attaches on the sides into small aluminum 
tracks.  It can be be motorized or hand-crank style.   Some models may 
snap the sides into small anchors placed into the deck providing  more 
shape ﬂexibility.   Provides safety (with water pumped off - cover pump), 
debris protection, and heat/chemical/water retention. 
COVER, HARD :  A cover which rests on the edge or coping of the spa 
or small pool.   Provides a barrier to 

In [31]:
vectordb.persist()

C:\Users\mbari\AppData\Local\Temp\ipykernel_11656\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
